In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data_dir = '/Users/kobets/beeline_data/'

In [4]:
data = pd.read_csv(data_dir + 'train.csv')
data.shape

(5454190, 10)

In [37]:
person_A = list(data['A'].unique())
person_B = list(data['B'].unique())
persons_all = set(person_A + person_B)
len(persons_all)

1211373

# Сопоставление пользователей с компаниями 

In [38]:
company_df_1 = data.sort('A').copy()
company_df_1.drop(labels=['B', 'x_B', 'c_AB', 'd_AB', 'c_BA', 'd_BA', 's_AB', 's_BA'], axis=1, inplace=True)
company_df_1.columns = ['person_id', 'company_id']
company_df_1.drop_duplicates(inplace=True)
company_df_1.shape

/Users/kobets/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


(973629, 2)

In [39]:
company_df_2 = data.sort('B').copy()
company_df_2.drop(labels=['A', 'x_A', 'c_AB', 'd_AB', 'c_BA', 'd_BA', 's_AB', 's_BA'], axis=1, inplace=True)
company_df_2.columns = ['person_id', 'company_id']
company_df_2.drop_duplicates(inplace=True)
company_df_2.shape

/Users/kobets/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


(910674, 2)

In [40]:
company_df = pd.concat([company_df_1, company_df_2]).sort('person_id')
company_df.drop_duplicates(inplace=True)
company_df.shape[0]

/Users/kobets/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


1211810

In [42]:
user_comp_counts = company_df['person_id'].value_counts()
bad_user_list = [i for i in persons_all if user_comp_counts[i] > 1]
len(bad_user_list)

437

In [341]:
for u in bad_user_list:
    d = {i:0 for i in set(company_df[company_df['person_id'] == u]['company_id'])}
    for l in ['A', 'B']:
        co = data[data['%s' % l] == u]['x_%s' %l].value_counts()
        if len(co) > 0:
            d[co.argmin()] = d[co.argmin()] + co.min()
            d[co.argmax()] = d[co.argmax()] + co.max()
    
    company_df.set_value(company_df['person_id'] == u, 'company_id', max(d, key=d.get))

In [342]:
company_df.drop_duplicates(inplace=True)
company_df.to_csv(data_dir + 'companies.csv', index=False, header=False)

with open(data_dir + 'companies_desc.txt', 'w') as f:
    for i in company_df.columns:
        f.write(i + '\n')

In [29]:
names = [i.rstrip('\n') for i in open(data_dir + 'companies_desc.txt')]
company_df = pd.read_csv(data_dir + 'companies.csv', header=None, names=names)
company_df.head()

,person_id,company_id
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3


# Нормализация исходных данных

In [83]:
X = data.copy()

X['person_A'] = X[['A', 'B']].min(axis=1)
X['person_B'] = X[['A', 'B']].max(axis=1)

X.set_value(X['A'] == X['person_A'], 'company_A', X['x_A'])
X.set_value(X['A'] != X['person_A'], 'company_A', X['x_B'])
X.set_value(X['B'] == X['person_B'], 'company_B', X['x_B'])
X.set_value(X['B'] != X['person_B'], 'company_B', X['x_A'])

X.set_value(X['A'] == X['person_A'], 'messages_A', X['c_AB'])
X.set_value(X['A'] != X['person_A'], 'messages_A', X['c_BA'])
X.set_value(X['B'] == X['person_B'], 'messages_B', X['c_BA'])
X.set_value(X['B'] != X['person_B'], 'messages_B', X['c_AB'])

X.set_value(X['A'] == X['person_A'], 'calls_A', X['d_AB'])
X.set_value(X['A'] != X['person_A'], 'calls_A', X['d_BA'])
X.set_value(X['B'] == X['person_B'], 'calls_B', X['d_BA'])
X.set_value(X['B'] != X['person_B'], 'calls_B', X['d_AB'])

X.set_value(X['A'] == X['person_A'], 'files_A', X['s_AB'])
X.set_value(X['A'] != X['person_A'], 'files_A', X['s_BA'])
X.set_value(X['B'] == X['person_B'], 'files_B', X['s_BA'])
X.set_value(X['B'] != X['person_B'], 'files_B', X['s_AB'])

X.drop(labels=['A', 'B', 'x_A', 'x_B', 'c_AB', 'c_BA', 'd_AB', 'd_BA', 's_AB', 's_BA'], axis=1, inplace=True)

In [84]:
X.head()

,person_A,person_B,company_A,company_B,messages_A,messages_B,calls_A,calls_B,files_A,files_B
0,666804,941235,1.0,0.0,1.0,1.0,22.0,20.0,0.0,0.0
1,367223,604328,0.0,1.0,5.0,0.0,1364.0,0.0,0.0,0.0
2,932768,977234,0.0,0.0,1.0,0.0,168.0,0.0,0.0,0.0
3,395101,677107,0.0,1.0,1.0,0.0,160.0,0.0,0.0,0.0
4,102647,250712,0.0,0.0,3.0,0.0,456.0,0.0,0.0,0.0


## исправление id компаний

In [92]:
names = [i.rstrip('\n') for i in open('/Users/kobets/beeline_data/companies_desc.txt')]
companies_df = pd.read_csv('/Users/kobets/beeline_data/companies.csv', header=None, names=names)
companies_df.head()

,person_id,company_id
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3


In [86]:
for u in bad_user_list:
    comp_id = int(companies_df[companies_df['person_id'] == u]['company_id'])
    X.set_value(X['person_A'] == u, 'company_A', comp_id)
    X.set_value(X['person_B'] == u, 'company_B', comp_id)

In [88]:
X = X.astype(int)
X.to_csv('/Users/kobets/beeline_data/train_new.csv', index=False, header=False)

with open(data_dir + 'train_new_desc.txt', 'w') as f:
    for i in X.columns:
        f.write(i + '\n')

# Подсчет связей между компаниями

In [242]:
X['company_min'] = X[['company_A', 'company_B']].min(axis=1)
X['company_max'] = X[['company_A', 'company_B']].max(axis=1)
X['companies'] = [tuple(x) for x in X[['company_min', 'company_max']].values]

In [246]:
companies_links = dict()
for i in range(5):
    for j in range(i, 5):
        companies_links[(i, j)] = sum(X['companies'] == (i, j))
        
companies_links

{(0, 0): 1295327,
 (0, 1): 1381139,
 (0, 2): 1118652,
 (0, 3): 1609742,
 (0, 4): 23187,
 (1, 1): 2805,
 (1, 2): 270,
 (1, 3): 3228,
 (1, 4): 73,
 (2, 2): 1932,
 (2, 3): 2727,
 (2, 4): 36,
 (3, 3): 14732,
 (3, 4): 110,
 (4, 4): 110}

In [27]:
companies_links_df = pd.DataFrame(sorted([(i[0][0], i[0][1], i[1]) for i in list(companies_links.items())]),
                  columns=['company_A', 'company_B', 'companies_links'])

In [30]:
company_size = {i:company_df.company_id.value_counts()[i] for i in range(5)}
company_size

{0: 350577, 1: 264077, 2: 239175, 3: 353920, 4: 3624}

In [31]:
# максивальеное возможное число связей между парой компаний

for i in range(5):
    for j in range(i,5):
        companies_links_df.set_value((companies_links_df.company_A == i) & (companies_links_df.company_B == j),
                                    'companies_potential', company_size[i] * company_size[j])

for i in range(5):
    companies_links_df.set_value((companies_links_df.company_A == i) & (companies_links_df.company_B == i),
                                    'companies_potential', company_size[i]*(company_size[i] - 1)/2)

In [43]:
companies_links_df

,company_A,company_B,companies_links,companies_potential
0,0,0,1295327,61451941176
1,0,1,1381139,92579322429
2,0,2,1118652,83849253975
3,0,3,1609742,124076211840
4,0,4,23187,1270491048
5,1,1,2805,34868198926
6,1,2,270,63160616475
7,1,3,3228,93462131840
8,1,4,73,957015048
9,2,2,1932,28602220725


In [32]:
companies_links_df = companies_links_df.astype(int)
companies_links_df.to_csv('/Users/kobets/beeline_data/companies_links.csv', index=False, header=False)

with open(data_dir + 'companies_links_desc.txt', 'w') as f:
    for i in companies_links_df.columns:
        f.write(i + '\n')

In [42]:
names = [i.rstrip('\n') for i in open('/Users/kobets/beeline_data/companies_links_desc.txt')]
companies_links_df = pd.read_csv('/Users/kobets/beeline_data/companies_links.csv', header=None, names=names)

In [13]:
companies_links_df['a1'] = companies_links_df.companies_links / (companies_links_df.companies_potential - companies_links_df.companies_links)

In [15]:
scaler = StandardScaler()
companies_links_df['a2'] = scaler.fit_transform(companies_links_df['a1'])

/Users/kobets/miniconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/kobets/miniconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
